<a href="https://colab.research.google.com/github/mardzyuba/Faker/blob/master/lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Использовать датасет с текущего урока и построить сегментированную воронку конверсии пользователей интернет-магазина по полу.

In [ ]:
import pandas as pd


In [6]:
user_table = pd.read_csv('/content/sample_data/user_table.csv')

user_table.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [7]:
user_table.shape


(90400, 4)

In [8]:
home_page = pd.read_csv('/content/sample_data/home_page_table.csv')
home_page.head()


,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [9]:
search_page = pd.read_csv('/content/sample_data/search_page_table.csv')
payment_page = pd.read_csv('/content/sample_data/payment_page_table.csv')
payment_confirmation = pd.read_csv('/content/sample_data/payment_confirmation_table.csv')

In [10]:
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [11]:
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [12]:
df.shape

(90400, 8)

In [13]:
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [15]:
summary = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().reset_index()
summary= summary.rename(columns = {'index':'stage', 0:'number'})
summary.head()

,stage,number
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation,452


In [16]:
df_sex = df.groupby(['sex'])[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()
df_sex.columns.name = None
df_sex = df_sex.rename(columns = {'index':'Total'})
df_sex.head()

,Total,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [17]:
from plotly import graph_objects as go

fig = go.Figure()


fig.add_trace(go.Funnel(
    y = df_sex['Total'],
    x = df_sex['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

fig.add_trace(go.Funnel(
    y = df_sex['Total'],
    x = df_sex['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии пользователей интернет-магазина по полу',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

## 2. Создать 2 новые фичи на основе колонки “date”: месяц и день недели 

In [35]:
import datetime

In [33]:
df['date'] = df['date'].apply(pd.to_datetime)


In [37]:
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day_name()
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day
0,450007,2015-02-28,Desktop,Female,1,0,0,0,February,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,January,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,April,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,February,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,January,Thursday


### 3. Определить самый топовый по продажам месяц и день недели с помощью базовых методов агрегации (sum или count).

In [47]:
top_day = df.groupby('day')['payment_confirmation'].sum().idxmax()

print(f'Топовый день по продажам \n   {top_day}')

Топовый день по продажам 
   Monday


In [46]:
top_month = df.groupby('month')['payment_confirmation'].sum().idxmax()

print(f'Топовый месяц по продажам \n   {top_month}' )

Топовый месяц по продажам 
   January


### 4. Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с помощью одного из методов построения сводных таблиц (groupby, pivot_table или crosstab).

In [32]:
pd.pivot_table(df, index = ['sex'], columns = ['day'], aggfunc = [sum], values = ['payment_confirmation'])

sum                                                  
       payment_confirmation                                                  
day                  Friday Monday Saturday Sunday Thursday Tuesday Wednesday
sex                                                                          
Female                   26     43       36     35       31      40        30
Male                     28     34       34     27       35      29        24

Мужчины совершили большего всего покупок в Thursday
А женщины в Monday